In [ ]:
# new
from openai import OpenAI
import json

# Open the JSON file for reading
with open('./credentials.json', 'r') as file:
    # Parse the JSON data into a Python dictionary
    data = json.load(file)

In [ ]:
client = OpenAI(
  # api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_key = data['openAI_Key']
)

# print(client.api_key)

In [ ]:

def get_image_description(prompt):
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    print(completion.choices[0].message)
    return completion.choices[0].message.content


def generate_image(description):
    response = client.images.generate(
      model="dall-e-3",
      prompt=description,
      size="1024x1024",
      quality="standard",
      n=1,
    )
    image_url = response.data[0].url
    return image_url



### Example of generating an image based on the description
* Please generate a description of a poor (economically disadvantaged) person.
* Please generate a description of another poor (economically disadvantaged) person different from the previously generated ones. 

In [ ]:
# Please generate a description of a poor (economically disadvantaged) person.
user_prompt = input("What would you like to see an image of? Enter the description please. ")
description = get_image_description(user_prompt)
image_url = generate_image(description)
print("Generated Image URL:", image_url)



### Fairness monitor and enforcement. 

In [ ]:
import random

def process_text(text, func, parameter):
    return func(text, parameter)


def fairness_enforcement(items, header_description, tail_description, beta):

    
    if beta < len(items):
        return "The length of the items should at least be as large as the beta value"
    
    # Initialize the counters for each item with the value beta
    counters = {item: beta for item in items}

    first = True
    
    while True:

        enforcement_prompt = ""
        # Display current counters for transparency
        print("Current counters:", counters)

        # Check the condition from largest to smallest
        for k in range(beta, 0, -1):
            # Find items with a counter value of exactly k
            matching_items = [item for item, count in counters.items() if count == k]
            # If there are exactly k items with the counter value k
            if len(matching_items) == k:

                # Randomly select an item from those that match the condition
                replacement = random.choice(matching_items)
                final_input = replacement
                print("ENFORCEMENT: Enforcing '{}' based on the condition.".format(replacement))
                enforcement_prompt = header_description+replacement+tail_description             
                break


        prompt = "Please generate a description of another poor (economically disadvantaged) person different from the previously generated ones." + enforcement_prompt
        if first == True:
            prompt = "Please generate a description of a poor (economically disadvantaged) person." + enforcement_prompt

        
        
        # Call ChatGPT and DALLE
        description = get_image_description(prompt)
        image_url = generate_image(description)
        print("Generated Image URL:", image_url)

        # Get user input and ensure it is within the list of items
        user_input = input("Enter an item from the list {} or type 'EXIT' to quit: ".format(items))
        
        # Check for exit condition
        if user_input.upper() == "EXIT":
            print("Exiting the program.")
            break
        
        # Ensure the input is within the list of items
        while user_input not in items:
            user_input = input("Please enter a valid item from the list {}, or type 'EXIT' to quit: ".format(items))
            if user_input.upper() == "EXIT":
                print("Exiting the program.")
                return
        
        # Reset the counter of the final input to beta and decrement others by 1
        for item in counters:
            if item == user_input:
                counters[item] = beta
            else:
                counters[item] -= 1

        first = False



#### Ensuring gender fairness

In [ ]:
# Example usage (gender)
items = ["male", "female"]
beta = 3
header_description = " The person has a strong "
tail_description = " looking."
fairness_enforcement(items, header_description, tail_description, beta)

#### Ensuring demographics fairness

In [ ]:
# Example usage (demographic description)
items = ["Asian", "Caucasian", "African American", "Latino"]
beta = 6
header_description = ". The person has a strong "
tail_description = " looking."
# fairness_enforcement(items, header_description, tail_description, beta)